<a href="https://colab.research.google.com/github/TanJianWu/MaxKB/blob/main/Copy_of_RAG%E6%A1%88%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 代码说明

使用 LangChain、Weaviate 和 OpenAI 的 API 来实现一个基于检索增强生成（Retrieval-Augmented Generation, RAG）的方法，用于从文档中回答用户提出的问题。

**1.安装并导入模块**

* LangChain 库：提供了一组工具和模块，用于处理文档加载、分块、向量存储、提示生成、LLM（大语言模型）调用等任务。
* Weaviate：一种用于管理和搜索向量数据的向量数据库。
* Google Colab 的 userdata：用于安全地管理和存储用户的 API 密钥。


In [ ]:
!pip install langchain openai weaviate-client
!pip install -U langchain-community
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 368.3/368.3 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/

In [ ]:
import weaviate
from weaviate.embedded import EmbeddedOptions
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import DirectoryLoader
from google.colab import userdata

/usr/local/lib/python3.10/dist-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.0 at grpc_health/v1/health.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 数据处理

**2.加载数据**
* DirectoryLoader：用于加载指定目录中的文本文件，这里只加载 .txt 格式的文件。
* documents：加载的文档内容。

In [ ]:
#加载数据
loader = DirectoryLoader('/content/drive/MyDrive/Colab Notebooks/docs', glob="*.txt",loader_cls=TextLoader)
documents = loader.load()
print(documents)

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/docs/3.98万L4无人车来了！卷出行业新低，1小时卖掉半年产能.txt'}, page_content='3.98万L4无人车来了！卷出行业新低，1小时卖掉半年产能\n智能车参考\n·2024-06-29 14:37\n关注\n苏州L4玩家九识打造。\n3.98万起，最便宜无人车来了。\n\n不到4万元，实现L4级自动驾驶，能送快递、生鲜和外卖。\n\n一经推出，产品直接爆单，1小时卖掉半年产能。\n\n打造爆款无人车的，是来自苏州的**九识智能，创立仅三年，产品服务已覆盖上百座城市，累计送单数破亿。\n\n3.98万起，最便宜的无人车来了\n九识智能首场新品发布会，实际是发布了一个无人配送“车队”：\n\n根据客户对装载方式和空间的不同需求，推出了Z2，Z5 2024，Z8和Z10等多款产品，产品数字即可装载的空间大小，覆盖2m³-10m³区间，价格如下：\n\n\n\n这里面有的是新推出的产品，比如Z10，售价8.98万元，车长超4米，续航210km，满载重量1.5吨。\n\n\n\n也有老款升级换代，比如Z5(图为板车车型)，售价4.98万元，也是九识过去的主力车型：\n\n\n\n新Z5主要是对外观、功能和算法进行了升级：\n\n外观方面，车身稍微改动以降低风阻实现更长续航。\n\n同时将传感器融入车身，能适应更窄的道路。\n\n功能方面，车头换了新的大灯，光照范围更大，车身提示灯也更容易了解车况，车内也安装了监控。\n\n\n\n作为主力车型，Z5的售价还不到5万元，相比人工成本要低的多，发布会现场有客户透露“省钱50%都不止”。\n\n九识为什么能把无人车的价格做的这么低？\n\n对此，九识表示，主要通过两个手段：\n\n产品开发上，在保证产品功能不受影响的前提下，积极寻求更低成本的硬件替代。\n\n运营销售上，追求量的扩大，摊薄成本，实现“薄利多销”。\n\n\n\n售价虽然不高，硬核配置也不少：\n\n英伟达双Orin，提供512TOPS的算力。\n\n4颗激光雷达+5个毫米波雷达+12个超声波雷达+14个摄像头，感知范围超300m。\n\n说完硬件配置，再来看看软件算法：\n\n全新架构ZOE 2.0升级

**3.数据分块**
* CharacterTextSplitter：用于将文档分成较小的块。chunk_size=1024 表示每个块的最大字符数为 1024，chunk_overlap=128 表示相邻块之间有 128 字符的重叠。
* chunks：分割后的文档块列表。

In [ ]:
#数据分块
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
chunks = text_splitter.split_documents(documents)
print(chunks)

[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/docs/3.98万L4无人车来了！卷出行业新低，1小时卖掉半年产能.txt'}, page_content='3.98万L4无人车来了！卷出行业新低，1小时卖掉半年产能\n智能车参考\n·2024-06-29 14:37\n关注\n苏州L4玩家九识打造。\n3.98万起，最便宜无人车来了。\n\n不到4万元，实现L4级自动驾驶，能送快递、生鲜和外卖。\n\n一经推出，产品直接爆单，1小时卖掉半年产能。\n\n打造爆款无人车的，是来自苏州的**九识智能，创立仅三年，产品服务已覆盖上百座城市，累计送单数破亿。\n\n3.98万起，最便宜的无人车来了\n九识智能首场新品发布会，实际是发布了一个无人配送“车队”：\n\n根据客户对装载方式和空间的不同需求，推出了Z2，Z5 2024，Z8和Z10等多款产品，产品数字即可装载的空间大小，覆盖2m³-10m³区间，价格如下：\n\n这里面有的是新推出的产品，比如Z10，售价8.98万元，车长超4米，续航210km，满载重量1.5吨。\n\n也有老款升级换代，比如Z5(图为板车车型)，售价4.98万元，也是九识过去的主力车型：\n\n新Z5主要是对外观、功能和算法进行了升级：\n\n外观方面，车身稍微改动以降低风阻实现更长续航。\n\n同时将传感器融入车身，能适应更窄的道路。\n\n功能方面，车头换了新的大灯，光照范围更大，车身提示灯也更容易了解车况，车内也安装了监控。\n\n作为主力车型，Z5的售价还不到5万元，相比人工成本要低的多，发布会现场有客户透露“省钱50%都不止”。\n\n九识为什么能把无人车的价格做的这么低？\n\n对此，九识表示，主要通过两个手段：\n\n产品开发上，在保证产品功能不受影响的前提下，积极寻求更低成本的硬件替代。\n\n运营销售上，追求量的扩大，摊薄成本，实现“薄利多销”。\n\n售价虽然不高，硬核配置也不少：\n\n英伟达双Orin，提供512TOPS的算力。\n\n4颗激光雷达+5个毫米波雷达+12个超声波雷达+14个摄像头，感知范围超300m。\n\n说完硬件配置，再来看看软件算法：\n\n全新架构ZOE 2.0升级，采用雨雪滤波算法，让无人车能顶风冒雪配

**4.数据块存储**
* Weaviate Client：创建 Weaviate 客户端，用于与 Weaviate 数据库交互。EmbeddedOptions() 表示嵌入式选项，用于本地化操作。
* vectorstore：将分块后的文档存储到 Weaviate 中，并使用 OpenAI 的嵌入模型 (OpenAIEmbeddings) 将文本转换为向量，以便后续的检索。

In [ ]:
#数据块存储
client = weaviate.Client(
 embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
  client = client,
  documents = chunks,
  embedding = OpenAIEmbeddings(openai_api_key = userdata.get('openai_api_key')),
  by_text = False
)

INFO:weaviate-client:Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.26.1/weaviate-v1.26.1-Linux-amd64.tar.gz
INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 1136
<ipython-input-6-6afce0e33cc8>:9: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding = OpenAIEmbeddings(openai_api_key = userdata.get('openai_api_key')),


## RAG实现

**5.数据检索**
 * Retriever：将存储在 Weaviate 中的向量数据转化为可用于检索的对象。

In [ ]:
#数据检索
retriever = vectorstore.as_retriever()
print(retriever.get_relevant_documents("霸王茶姬香港店什么时候开业？店长薪酬是多少？") )

<ipython-input-7-7c307540d98f>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  print(retriever.get_relevant_documents("霸王茶姬香港店什么时候开业？店长薪酬是多少？") )


[Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/docs/8家消费公司拿到新钱;霸王茶姬高薪招香港店长;一季度快消品平均售价同比降1.5%｜创投大视野.txt'}, page_content='内地的新茶饮品牌和咖啡品牌纷纷进军香港，但高昂的人力成本和租金成本是个问题。据界面报道，香港洗碗工月薪能高达1.8万，因此霸王茶姬、蜜雪冰城开出的“高薪”在当地只能勉强称上正常水平。据香港媒体测算，蜜雪冰城旺角店需月售两万杯才能承担20万的月租。\n\n这已经对一些赴港扩张的内地品牌造成了阻碍。柠檬茶品牌“柠濛濛”、湘菜品牌“萝卜向南”等都已被曝关闭香港门店。背后的本质问题在于，内地的性价比优势是否能在香港本土实现，manner、星巴克、蜜雪冰城在香港的门店均设置了高于内地的定价。如今，香港和内地全面通关，“港人北上”成为颇为便利的风潮，野心勃勃的霸王茶姬需要更灵活的准备。\n\n•「小罐茶」创始人为营销套路致歉\n\n小罐茶希望尽快摆脱“强营销”的过去。\n\n6月25日，小罐茶创始人杜国楹在12周年发布上回应长期以来公众对“大师作”这一标签的争议。“大师作”的宣传语曾因是否由大师亲手做的问题引发诸多争议，杜国楹表示，企业广告语对大家造成了困扰，自己反思多年，郑重向大家道歉。\n\n如今，大众消费者对过度营销、智商税的抵触越来越高，小罐茶的日子也过得不太好。据界面报道，小罐茶近90天新开33家门店，新关59家；门店总数由2023年12月的1002家下滑到眼下的626家。按此计算，小罐茶在半年内关闭了376家门店。\n\n消费的大环境在发生变化，品牌也要顺势而变。道歉完毕后，杜国楹话锋一转，宣布小罐茶的135项制茶专利无偿向社会开源，霸王茶姬成为其专利开源的首位授予伙伴。此外，他们还将大力发展平价产品线“小罐茶园”，定位生活口粮茶，售价在100-300元/斤。\n\n•「大窑饮品」加入无糖茶大战\n\n大窑不只满足于汽水市场了。\n\n6月28日，大窑通过微信公众号宣布，设立新品牌“查元香”，同步推出金桂乌龙、白兰乌龙、乌龙茶3款茶饮新品。紧跟当下的无糖茶趋势，上述新品的亮点也是0糖、0脂肪、0能量，终端售价4-5元。在更早前的2022年，大窑已经启用过查元香品牌，在电商渠道售卖其果汁

**6.提示增强**
* template：定义了一个提示模板，用于指导 LLM 如何生成回答。模板中的 {question} 和 {context} 会在实际调用时被问题和上下文数据所替换。
* ChatPromptTemplate：用于将模板生成一个可用于 LLM 的提示对象。

In [ ]:
#提示增强
template = "你是一个问答机器人助手，请使用以下检索到的上下文来回答问题，如果你不知道答案，就说你不知道。问题是：{question},上下文: {context},答案是:"
prompt = ChatPromptTemplate.from_template(template)

**7.答案生成**
* ChatOpenAI：实例化 OpenAI 的 GPT-3.5 模型，用于生成基于上下文的答案。temperature=0 表示生成的答案会更加确定性，减少随机性。
* rag_chain：定义了一个处理链，依次进行数据检索、提示生成、答案生成以及最终的输出解析。
* {"context": retriever, "question": RunnablePassthrough()}：定义了 RAG 链的输入部分，retriever 用于检索上下文，RunnablePassthrough 用于直接传递用户的问题。
*	| prompt | llm | StrOutputParser()：表示链式调用。prompt 生成输入给 LLM 的提示，llm 生成答案，StrOutputParser() 将答案解析成字符串格式。

In [ ]:
#答案生成
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0,openai_api_key = userdata.get('openai_api_key'))

rag_chain = (
  {"context": retriever, "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

<ipython-input-9-ac404246773a>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0,openai_api_key = userdata.get('openai_api_key'))


In [ ]:
# query = "霸王茶姬香港店什么时候开业？什么地方？店长薪酬是多少？"
query = "今年英超冠军是谁？多少分"
res=rag_chain.invoke(query)
print(f'答案：{res}')

答案：今年英超冠军是曼城队，他们获得了91分。
